In [ ]:
# for more powerfull parameter construction
import itertools
import functools
import collections

# math library
import numpy as np

# data analysis librarz
import pandas as pd
#plotting library
import matplotlib.pyplot as plt
# display images in the notebook %matplotlib notebook makes them interactive!
%matplotlib inline

# for foldernames, timing etc
import sys
from datetime import datetime
import time

# show all resoruces available via VISA
import visa
rm = visa.ResourceManager()
print(rm.list_resources())
rm.close()

# import the actual library for the Tester
from agilentpyvisa.B1500 import *

# hide all internal logging. Set this to logging.INFO to see the commands being sent 
import logging
exception_logger.setLevel(logging.INFO)
write_logger.setLevel(logging.INFO)
query_logger.setLevel(logging.WARN)

In [ ]:
# connect to tester
b15= B1500('GPIB1::17::INSTR')
# print all connected instruments with channel numbers
for c in b15.sub_channels:
    print(c,b15.slots_installed[int(str(c)[0])].name)
b15.default_check_err=False

To make creating your own measuring script easier, here are some starting helper functions and definitions.

In [ ]:
# define channel numbers for use
SMU1=2 # B
SMU2=3 # C
SMU3=4 # D
SMU4=5 # E

# define setup functions with sane defaults for Memristor testing.

def get_pulse(base, peak, width,count=1, lead_part=0.8, trail_part=0.8, loadZ=1e6,gate_voltage=1.85,
              ground=SMU2,channel=101,gate=SMU3):
    """
    Defines a SPGU setup based on the given parameters
    The setup assumes we use a transistor with
    smu3->gate, 
    bottom_electrode->drain, 
    source->ground(SMU2 by default.)
    """
    mspgu=SPGU(base, peak,width, loadZ=loadZ, pulse_leading=[lead_part*width], pulse_trailing=[trail_part*width],condition=count )
    inp_channel=Channel(number=channel,spgu=mspgu)
    ground_channel=Channel(number=ground,dcforce=DCForce(Inputs.V,0,.1))
    gate_channel = Channel(number=gate,dcforce=DCForce(Inputs.V,gate_voltage,.1))
    spgu_test=TestSetup(channels=[ground_channel, gate_channel, inp_channel,],spgu_selector_setup=[(SMU_SPGU_port.Module_1_Output_2,SMU_SPGU_state.connect_relay_SPGU)])
    return (spgu_test,mspgu, inp_channel, ground_channel, gate_channel)

def get_Vsweep(start, stop, steps, compliance=300e-6,
               measure_range=MeasureRanges_I.full_auto,gate_voltage=1.85, ground=SMU2):
    """
    Defines a Sweep setup based on the given parameters
    The setup assumes we use a transistor with
    smu3->gate, 
    bottom_electrode->drain, 
    source->ground(SMU2 by default. We also measure here.SMU1 if you do not want to use the transistor for current limiting.)
    """
    swep_smu=b15.slots_installed[b15._B1500__channel_to_slot(3)]
    in_range=swep_smu.get_mincover_V(start,stop)
    
    sweep_measure=MeasureStaircaseSweep(Targets.I,range=measure_range, side=MeasureSides.current_side)
    
    sweep = StaircaseSweep(Inputs.V,InputRanges_V.full_auto,start,stop,steps,compliance, auto_abort=AutoAbort.disabled)
    
    inp_channel=Channel(number=SMU4,staircase_sweep=sweep, measurement=sweep_measure)
    
    ground_channel=Channel(number=ground,
                           dcforce=DCForce(Inputs.V,0,compliance),
                          )
    
    gate_channel = Channel(number=SMU3,dcforce=DCForce(Inputs.V,gate_voltage,.1),
                          )
    
    sweep_test=TestSetup(channels=[gate_channel,ground_channel,inp_channel],
                         spgu_selector_setup=[(SMU_SPGU_port.Module_1_Output_2,SMU_SPGU_state.connect_relay_SMU)],
                        output_mode=OutputMode.with_primarysource,
                        format=Format.ascii13_with_header_crl, filter=Filter.enabled)
    return (sweep_test,sweep, inp_channel, ground_channel, gate_channel)


# some very basic analysis functions

def get_R(d, current_column='EI', voltage_column='EV'):
    """
    Takes in padnas.DataFrame and optional column labels, returns average Resistance
    """
    R = d[voltage_column]/d[current_column]
    # remove all infinite resistances
    R=R.replace([np.inf, -np.inf], np.nan).dropna()
    return R.abs().mean()



def plot_output(out, t='line',up='b',down='r', voltage_column='EV', current_column='EI'):
    """ Show bat plot of voltage sweep, with different colours for up and down sweep"""
    lout = out[[voltage_column, current_column]]
    lout=lout[lout.applymap(lambda x: not np.isnan(x)).all(axis=1)] # remove NaN values
    y=lout[current_column].abs()*1e6 # scale up current from microns
    x=np.array(lout[voltage_column]) 
    half = lout[voltage_column].abs().idxmax() # find peak of sweep for up/down plot
    if t=='line':
        plt.plot(x[:half],y[:half], color=up,marker='o')
        plt.plot(x[half:],y[half:], color=down,marker='o')
    elif t=='scatter':
        plt.scatter(x[:half],y[:half], color=up,marker='o')
        plt.scatter(x[half:],y[half:], color=down,marker='o')
    plt.ylabel('Ground current in uA')
    plt.xlabel('Voltage forced in V')
    plt.autoscale()
    
# test functions
assert(get_pulse(0,1,0.5))  # base 0, peak 1V, width 0.5 s
assert(get_Vsweep(0,1,50))  # start 0, stop 1, 50 steps

With the setup functions, we define some basic procedures, which make up the vocabulary for our test

In [ ]:
# base abstract procedures

def sweep(stop, steps, compliance=300e-6,start=0,mrange=MeasureRanges_I.full_auto,
          gate=1.85, plot=True, up='b',down='r', ground=SMU2, stats=True):
    """ Create and immediately perform a sweep, optionally plot the data and/or show statistics"""
    b15.set_SMUSPGU_selector(SMU_SPGU_port.Module_1_Output_1,SMU_SPGU_state.connect_relay_SMU)
    set_setup,set, _ ,ground_channel,gate_channel=get_Vsweep(start=start,stop=stop,steps=steps, compliance=compliance,
                                                                     measure_range=mrange,gate_voltage=gate, ground=ground)
    ret,out =b15.run_test(set_setup, force_wait=True, auto_read=True,force_new_setup=True)
    out,series_dict,raw =out
    if plot:
        plot_output(out, up=up,down=down)
    if stats:
        print(out.describe())
    return out

def pulse(p_v, width, slope, gate=SMU3, gate_voltage=1.85,ground=SMU2,loadZ=1e6):
    reset_pulse_setup, reset_pulse,_,_,_= get_pulse(0,p_v,width,
                                                    lead_part=slope,trail_part=slope,gate=gate,gate_voltage=gate_voltage,
                                                    ground=ground,loadZ=loadZ
                                                   )
    b15.set_SMUSPGU_selector(SMU_SPGU_port.Module_1_Output_1,SMU_SPGU_state.connect_relay_SPGU)
    b15.run_test(reset_pulse_setup,force_wait=True,force_new_setup=True)
    b15.set_SMUSPGU_selector(SMU_SPGU_port.Module_1_Output_1,SMU_SPGU_state.connect_relay_SMU)

    
    
# some more concrete cases, with the defaults we use initially
def read(start=200e-6,stop=350e-6, steps=51,mrange=MeasureRanges_I.uA100_limited,
         gate=1.85, plot=True, print_R=True, stats=True):
    """
    A quick sweep to estimate the current Resistance of the DUT
    """
    out = sweep(start=start,stop=stop,steps=steps,mrange=mrange,gate=gate,plot=plot,stats=stats)
    return out

def checkR(start=200e-6,stop=350e-6, steps=51,mrange=MeasureRanges_I.uA100_limited,  gate=1.85):
    """
    Perform a read and calculate the mean, throw everything else away
    """
    out= read(start=start,stop=stop, steps=steps,mrange=mrange,gate=gate, plot=False,stats=False)
    R= get_R(out)
    return R


def form(forming_v, steps, compliance,mrange=MeasureRanges_I.full_auto, gate=1.85):
    """
    Initial form sweep for the DUT
    """
    return sweep(forming_v, steps, compliance,mrange=mrange, gate=gate, plot=True, up='c', down='m')
    
    
def set_sweep(set_v, steps, compliance,mrange=MeasureRanges_I.full_auto, gate=1.85, plot=True):
    """
    DC Set, with SMU2 as ground -> transistor limits current in LRS after set
    """
    return sweep(set_v, steps, compliance,mrange=mrange, gate=gate, plot=plot, up='c', down='m')
    
def reset_sweep(reset_v, steps, compliance,mrange=MeasureRanges_I.full_auto, gate=1.85, plot=True):
    """
    DC reset, with SMU1 as ground -> no transistor limiting
    """
    return sweep(reset_v, steps, compliance,mrange=mrange, gate=gate, plot=plot, ground=SMU1)

# uncomment to test the functions, but beware, they do use the tester
#assert(read(stats=False,plot=False) is not None)
#assert(form(0,10,0.1) is not None)
#assert(set_sweep(0,10,0.1) is not None)
#assert(reset_sweep(0,10,0.1) is not None)

We can then also build more complex testing procedures from these building blocks

In [ ]:
def pulse_cycle(rV=-0.5,sV=0.8,width=1e-3,slope=0.8,pr=True,gateReset=1.85,gateSet=1.85,loadZreset=1e6,loadZset=1e6):
    """
    Perform a reset/set cycle using the spgu, checking the change of R with a read pre reset, post reset and post set
    """
    pre_reset =checkR()
    pulse(rV, width, slope,ground=SMU1,gate_voltage=gateReset,loadZ=loadZreset)
    #post_reset = checkR(pr=pr)
    pre_set = checkR()
    pulse(sV, width,slope,gate_voltage=gateSet,loadZ=loadZset)
    post_set=checkR()
    #return (pre_reset, post_reset,pre_set,post_set)
    return (pre_reset, pre_set, post_set)

def pulse_iter(rV=-0.5,sV=0.8,width=1e-3,slope=0.8,max_iter=100,pr=True, abort_break=True,gateReset=1.85, gateSet=1.85,
              loadZreset=1e6,loadZset=1e6):
    """
    An iterator which performs max_iter pulse cycles,
    or optionally until there is no order of magnitude change between set& reset
    """
    i=0
    while True:
        #if i%100 == 0:
        #    print('Iteration ',i)
        #pR,poR,pS,poS = pulse_cycle(rV,sV,width,slope,pr=pr,gateReset=gateReset, gateSet=gateSet)
        pR,pS,poS = pulse_cycle(rV,sV,width,slope,pr=pr,gateReset=gateReset, gateSet=gateSet
                                ,loadZreset=loadZreset,loadZset=loadZset)
        
        yield (pR,pS,poS)
        if (abort_break  and np.log10(poR/poS) < 1) or i>=max_iter:
            
            break
        else:
            i+=1
    raise StopIteration
    

def check_width(w,it,rV=-0.5,sV=0.8,slope=0.8,plot=True, abort_break=True,gateReset=1.85,gateSet=1.85,loadZreset=1e6,loadZset=1e6):
    """
    Build a pulse_iter from given parameters and construct a dataFrame from it
    """
    c3=pd.DataFrame(np.fromiter(pulse_iter(width=w,max_iter=it,rV=rV,sV=sV,slope=slope,pr=False,
                                           abort_break=abort_break,
                                           gateReset=gateReset,
                                          gateSet=gateSet,
                                          loadZreset=loadZreset,
                                           loadZset=loadZset),
                      dtype=[('preReset',np.float),('preSet',np.float),('postSet',np.float)]))
    print(c3.describe())
    if plot:
        c3.plot()
    return c3

And even more complex tests with automated logging, estimation when we are done and some basic SI unit handling

In [ ]:
def unit(u):
    if u[0]=='m':
        return 1e-3
    elif u[0]=='u':
        return 1e-6
    elif u[0]=='n':
        return 1e-9
    elif u[0]=='p':
        return 1e-12
    elif u[0]=='f':
        return 1e-15
    elif u[0]=='k':
        return 1e3
    elif u[0]=='M':
        return 1e6
    elif u[0]=='G':
        return 1e9
    else:
        return 1

def pyramid_voltage(voltages=[],gate_voltages_reset=[1850],
                    gate_voltages_set=[1850],voltage_si='mV',widths=[50],time_si='us',slope=0.8,times=10, abort_break=True,
                   loadZreset=1e6,loadZset=1e6):
    """
    Test a list of combinations of reset/set voltages and gate voltages 
    """
    datas=[]
    iteration=0
    end=len(widths)*len(voltages)*len(gate_voltages_reset)
    time_per_run=None
    for w in widths:
        for gR,gS in zip(gate_voltages_reset,gate_voltages_set):
            for rV,sV in voltages:
                start=time.time()
                print("Iteration ",
                      iteration,
                      " of ",
                      end)
                if time_per_run:
                    print(
                          'estimated end',
                          time.strftime('%H:%M:%S',
                                        time.localtime(start+time_per_run*(end-iteration))
                                        ),
                          " time per run:",
                          time_per_run,
                          "seconds"
                         )
                d=None
                width=w*unit(time_si)
                resetV=rV*unit(voltage_si)
                setV=sV*unit(voltage_si)
                gateReset=gR*unit(voltage_si)
                gateSet=gS*unit(voltage_si)
                print("Running:")
                print("width:",width)
                print('lead/trail:',slope*width)
                print('resetV:',resetV)
                print('setV:',setV)
                print('gateSet:',gateSet)
                print('gateReset:',gateReset)
                print('loadZset:',loadZset)
                print('loadZreset:',loadZreset)

                try:
                    d=check_width(width, times,rV=resetV,sV=setV,slope=slope,plot=False,
                                  abort_break=abort_break,gateReset=gateReset,
                                 gateSet=gateSet,loadZreset=loadZreset,loadZset=loadZset)
                except BaseException as e:
                    pass
                if d is not None:
                    d['rV']=    pd.Series((resetV for _ in range(len(d))),index=d.index)
                    d['sV']=    pd.Series((setV for _ in range(len(d))),index=d.index)
                    d['gateReset']=  pd.Series((gateReset for _ in range(len(d))),index=d.index)
                    d['gateSet']=  pd.Series((gateSet for _ in range(len(d))),index=d.index)
                    d['width']= pd.Series((width for _ in range(len(d))),index=d.index)
                    d['slope']= pd.Series((slope for _ in range(len(d))),index=d.index)
                    d['loadZset']= pd.Series((loadZset for _ in range(len(d))),index=d.index)
                    d['loadZreset']= pd.Series((loadZreset for _ in range(len(d))),index=d.index)
                    datas.append(d)
                    # save the intermittent results in case we need to termiante early
                    d.to_csv('{timestamp}_pyramid_run{run_id}_w_{width}_rV_{rV}__sV_{sV}__rgV_{rgV}__sgV_{sgV}_slope_{slope}percent.csv'.format(
                        timestamp=datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
                        run_id=iteration,
                        width=format(width/unit(time_si))+time_si,
                        rV=str(rV)+voltage_si,
                        sV=str(rV)+voltage_si,
                        rgV=str(gR)+voltage_si,
                        sgV=str(gS)+voltage_si,
                        slope=slope/1e-2
                            ))
                iteration+=1
                time_per_run=time.time()-start
    if datas:
        data = pd.concat(datas)
        data.index=range(len(data))
        data.to_csv('{timestamp}_pyramid_{num_iters}runs_full.csv'.format(
                    timestamp=datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),num_iters=end))
        return data

# Sample tests

Below is how I used the setup above when testing, together with the results

In [ ]:
form_sweep= plt.figure(figsize=[10,5])
f=form(3,100,10e-3, mrange=MeasureRanges_I.uA10_limited,gate=1.9)
f.to_csv("{}_form_3V.csv".format(datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
plt.autoscale()
checkR()

In [ ]:
iters=2
for i in range(iters):
    plt.figure()
    plt.hold(True)
    rt=reset_sweep(-1.5,100,5e-3, mrange=MeasureRanges_I.uA10_limited,gate=1.9)
    print('Reset')
    rt.to_csv("{}_reset{}_-1_5V.csv".format(
            datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),i)
             )
    print('HRS',checkR())
    print('Set')
    s=set_sweep(1.5,100,10e-3, mrange=MeasureRanges_I.uA10_limited,gate=1.9)
    s.to_csv("{}_set{}_1V.csv".format(
            datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),i)
            )
    print('LRS',checkR())
    plt.hold(False)

In [ ]:
plt.hold()

In [ ]:
def norm(d):
    parameters=['sV','rV','gateSet','gateReset','slope','width']
    data=[c for c in d.columns if c not in parameters ]
    d_par = d[parameters]
    d_dat = d[data]
    d_dat_norm= (d_dat-d_dat.mean())/(d_dat.max()-d_dat.min())
    return pd.merge(d_dat_norm,d_par,left_index=True,right_index=True)
    

In [ ]:
baseV = (-800,500)  #mV
basegate=1900 #mV
basewidth=500  #us

r,s=baseV
cycles_per_test=10

percentages=list(reversed(range(100,101,5)))
gate_voltages_set=[basegate]#* p/100 for p in percentages]
gate_voltages_reset=gate_voltages_set
loadZset=10
loadZreset=10

rL=[r*i/100 for i in percentages]
sL=[s*i/100 for i in percentages]
voltage_pyr=list(itertools.product(rL,sL))
widths=[basewidth,]
print("Num_it",len(widths)*len(voltage_pyr)*len(gate_voltages_set))
print("Widths",widths,'us')
print("Percentages",percentages,'%')
print("Gate Voltages",list(zip(gate_voltages_reset,gate_voltages_set)),'mV')
print("Pulse Voltages",voltage_pyr,'mV')
print('loadZset:',loadZset,'Ohm')
print('loadZreset:',loadZreset,'Ohm')
if True:
    vpyr=pyramid_voltage(voltage_pyr,
                     widths=widths,
                     time_si='us',
                     times=cycles_per_test,
                     gate_voltages_set=gate_voltages_set,
                     gate_voltages_reset=gate_voltages_reset,
                     voltage_si='mV',
                     abort_break=False,
                        loadZset=loadZset,
                        loadZreset=loadZreset,)

In [ ]:
vpyr_norm=norm(vpyr)

In [ ]:
%matplotlib inline
to_exclude=['gateReset','gateSet','slope','width']
vpyr.ix[:,vpyr.columns.difference(to_exclude)].plot(logy=[True,True], secondary_y=['rV','sV','gateReset','gateSet','width','slope'],figsize=[10,5], title="The original pulse run")

if False:
    sortpyr=vpyr.sort_values(['sV','rV'],ascending=[False,True])
    sortpyr.index=range(len(sortpyr))
    sortpyr.ix[:,vpyr.columns.difference(to_exclude)].plot(logy=[True,True], secondary_y=['rV','sV','gateReset','gateSet','width','slope'],figsize=[10,5], title="Sorted by sV-desc then rV-asc")

    sortpyr2=vpyr.sort_values(['rV','sV'],ascending=[True,False])
    sortpyr2.index=range(len(sortpyr2))
    sortpyr2.ix[:,vpyr.columns.difference(to_exclude)].plot(logy=[True,True], secondary_y=['rV','sV','gateReset','gateSet','width','slope'],figsize=[10,5], title="Sorted by rV-asc then sV-desc")

if False:
    pd.tools.plotting.scatter_matrix(vpyr)

In [ ]:
%matplotlib inline
plt.figure(figsize=[10,10])
#plt.yscale('lin')
pd.tools.plotting.parallel_coordinates(vpyr_norm[[x for x in vpyr_norm.columns if x not in ['width','slope','gateReset']]],'gateSet')

In [ ]:
grouping_variables=['sV','rV',]
trivial=['width','slope','gateReset']
grps=vpyr_norm.groupby(grouping_variables)
#grps.plot()

In [ ]:
%matplotlib inline
for key,g in grps:
    
    g_sel=g[[x for x in g.columns if x not in grouping_variables+trivial]]
    plt.figure(figsize=[10,5])
    groupvar='gateSet'
    pd.tools.plotting.parallel_coordinates(g_sel,groupvar)
    #plt.yscale('log')
    plt.title(' '.join(['{}:{}'.format(l,v) for l,v in zip(grouping_variables,key)]+['groupedBy:{}'.format(groupvar)]))

    #.plot(logy=[True,True],
    #       figsize=[10,5], title='{}'.format({l:v for l,v in zip(grouping_variables,key)}))

grps.describe()

In [ ]:
grps[['postSet',]].var().plot(figsize=[10,5])